<a href="https://colab.research.google.com/github/FraGoTe/Analisis-Estadistico-Textos/blob/master/ClusteringHDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

#!rm -R dataset
#!mkdir -p dataset
#!wget https://raw.githubusercontent.com/githila/data/master/bbc.zip -P dataset

#!unzip dataset/bbc.zip -d dataset/bbc
#!rm dataset/bbc.zip

# HDP (Hierarchical Dirichlet Process)

## ¿Qué es?

Es un algoritmo que se puede emplear para clustering de texto, realmente su funcionalidad es el modelado de temas, donde un tema es modelado a partir de grupo de documentos.

## Librerías requeridas

- codecs
- nltk
- numpy
- functools
- re

Las siguientes librerías son para leer múltiples archivos. El algoritmo base HDP no las necesita
- os
- pathlib



### Imports


In [0]:
import nltk, re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Stopwords

Se define la lista de stopwords, en adición a las incluidas en _nltk.corpus.stopwords.words('english')_ list.
Se genera un conjunto para no tener palabras repetidas.


In [0]:
# stopwords_list = nltk.corpus.stopwords.words('english')
stopwords_list = "a,s,able,about,above,according,accordingly,across,actually,after,afterwards,again,against,ain,t,all,allow,allows,almost,alone,along,already,also,although,always,am,among,amongst,an,and,another,any,anybody,anyhow,anyone,anything,anyway,anyways,anywhere,apart,appear,appreciate,appropriate,are,aren,t,around,as,aside,ask,asking,associated,at,available,away,awfully,be,became,because,become,becomes,becoming,been,before,beforehand,behind,being,believe,below,beside,besides,best,better,between,beyond,both,brief,but,by,c,mon,c,s,came,can,can,t,cannot,cant,cause,causes,certain,certainly,changes,clearly,co,com,come,comes,concerning,consequently,consider,considering,contain,containing,contains,corresponding,could,couldn,t,course,currently,definitely,described,despite,did,didn,t,different,do,does,doesn,t,doing,don,t,done,down,downwards,during,each,edu,eg,eight,either,else,elsewhere,enough,entirely,especially,et,etc,even,ever,every,everybody,everyone,everything,everywhere,ex,exactly,example,except,far,few,fifth,first,five,followed,following,follows,for,former,formerly,forth,four,from,further,furthermore,get,gets,getting,given,gives,go,goes,going,gone,got,gotten,greetings,had,hadn,t,happens,hardly,has,hasn,t,have,haven,t,having,he,he,s,hello,help,hence,her,here,here,s,hereafter,hereby,herein,hereupon,hers,herself,hi,him,himself,his,hither,hopefully,how,howbeit,however,i,d,i,ll,i,m,i,ve,ie,if,ignored,immediate,in,inasmuch,inc,indeed,indicate,indicated,indicates,inner,insofar,instead,into,inward,is,isn,t,it,it,d,it,ll,it,s,its,itself,just,keep,keeps,kept,know,knows,known,last,lately,later,latter,latterly,least,less,lest,let,let,s,like,liked,likely,little,look,looking,looks,ltd,mainly,many,may,maybe,me,mean,meanwhile,merely,might,more,moreover,most,mostly,much,must,my,myself,name,namely,nd,near,nearly,necessary,need,needs,neither,never,nevertheless,new,next,nine,no,nobody,non,none,noone,nor,normally,not,nothing,novel,now,nowhere,obviously,of,off,often,oh,ok,okay,old,on,once,one,ones,only,onto,or,other,others,otherwise,ought,our,ours,ourselves,out,outside,over,overall,own,particular,particularly,per,perhaps,placed,please,plus,possible,presumably,probably,provides,que,quite,qv,rather,rd,re,really,reasonably,regarding,regardless,regards,relatively,respectively,right,said,same,saw,say,saying,says,second,secondly,see,seeing,seem,seemed,seeming,seems,seen,self,selves,sensible,sent,serious,seriously,seven,several,shall,she,should,shouldn,t,since,six,so,some,somebody,somehow,someone,something,sometime,sometimes,somewhat,somewhere,soon,sorry,specified,specify,specifying,still,sub,such,sup,sure,t,s,take,taken,tell,tends,th,than,thank,thanks,thanx,that,that,s,thats,the,their,theirs,them,themselves,then,thence,there,there,s,thereafter,thereby,therefore,therein,theres,thereupon,these,they,they,d,they,ll,they,re,they,ve,think,third,this,thorough,thoroughly,those,though,three,through,throughout,thru,thus,to,together,too,took,toward,towards,tried,tries,truly,try,trying,twice,two,un,under,unfortunately,unless,unlikely,until,unto,up,upon,us,use,used,useful,uses,using,usually,value,various,very,via,viz,vs,want,wants,was,wasn,t,way,we,we,d,we,ll,we,re,we,ve,welcome,well,went,were,weren,t,what,what,s,whatever,when,whence,whenever,where,where,s,whereafter,whereas,whereby,wherein,whereupon,wherever,whether,which,while,whither,who,who,s,whoever,whole,whom,whose,why,will,willing,wish,with,within,without,won,t,wonder,would,would,wouldn,t,yes,yet,you,you,d,you,ll,you,re,you,ve,your,yours,yourself,yourselves,zero,the,of,and,to,is,that,was,for,it,in,om,it,i,it,that,is,in,and,of,to,a,the".split(',');

# se convierten a conjunto
stopwords_list = set(stopwords_list)

# casos específicos
recover_list = {"wa":"was", "ha":"has"}

def is_stopword(w):
    return w in stopwords_list

### Lematización

In [0]:
wl = nltk.WordNetLemmatizer()

def lemmatize(w0):
    w = wl.lemmatize(w0.lower())
    if w in recover_list: return recover_list[w]
    return w

## Clase *Vocabulary*

Esta clase toma la lista de documentos y extrae las palabras. A cada palabra le asigna un ID. Dependiendo de los stopwords, no todas las palabras de los documentos se retienen para el análisis de los temas.

Lo más importante aquí es _self.vocas_ que es la lista de todas aquellas palabras retenidas (que no son stopwords).

Los métodos _term_to_id_ y _doc_to_id_ eliminan las palabras repetidas, de tal manera que cada palabra sólo se cuenta cuántas veces aparece en el documento.

In [0]:
class Vocabulary:
    def __init__(self, excluds_stopwords=False):
        self.vocas = []        # id para palabra
        self.vocas_id = dict() # palabra que corresponde a un id
        self.docfreq = []      # id para la frecuencia del documento
        self.excluds_stopwords = excluds_stopwords

    def term_to_id(self, term0):
        term = lemmatize(term0)
        if not re.match(r'[a-z]+$', term): return None
        if self.excluds_stopwords and is_stopword(term): return None
        try:  
            term_id = self.vocas_id[term]
        except:
            term_id = len(self.vocas)
            self.vocas_id[term] = term_id
            self.vocas.append(term)
            self.docfreq.append(0)
        return term_id

    def doc_to_ids(self, doc):
        l = []
        words = dict()
        for term in doc.split():
            id = self.term_to_id(term)
            if id != None:
                l.append(id)
                if not id in words:
                    words[id] = 1
                    self.docfreq[id] += 1 
                    #Cuenta en cuántos documentos aparece una palabra.
                    #Si aparece muy poco la remueve del vocabulario mediante cut_low_freq()
        if "close" in dir(doc): doc.close()
        return l

    def cut_low_freq(self, corpus, threshold=1):
        new_vocas = []
        new_docfreq = []
        self.vocas_id = dict()
        conv_map = dict()
        for id, term in enumerate(self.vocas):
            freq = self.docfreq[id]
            if freq > threshold:
                new_id = len(new_vocas)
                self.vocas_id[term] = new_id
                new_vocas.append(term)
                new_docfreq.append(freq)
                conv_map[id] = new_id
        self.vocas = new_vocas
        self.docfreq = new_docfreq

        def conv(doc):
            new_doc = []
            for id in doc:
                if id in conv_map: new_doc.append(conv_map[id])
            return new_doc
        return [conv(doc) for doc in corpus]

    def __getitem__(self, v):
        return self.vocas[v]

    def size(self):
        return len(self.vocas)

    def is_stopword_id(self, id):
        return self.vocas[id] in stopwords_list



## Clase *HDP*

### Imports

In [0]:
import numpy, codecs
from datetime import datetime
from numpy.random import choice
from numpy import *
import functools
import sys
import os
from pathlib import Path
sys.setrecursionlimit(10000)

import nltk
#nltk.download('wordnet')

### Clase _HDP_gibbs_sampling_

En el método __init__ method se definen todos los parámetros necesarios:.
- Número de temas a obtener.
- Los parámetros de las distribuciones alpha, beta y gamma (por default 0.5, 0.5 y 1.5 respectivamente) 
- La lista de documentos a analizar
- El vocabulario que se empleará


In [0]:
class HDP_gibbs_sampling:    
    def __init__(self, K0=10, alpha=0.5, beta=0.5,gamma=1.5, docs= None, V= None):
        self.maxnn = 1

        # el número de grupos los infiere mediante el número de Stirling de segundo orden
        self.alss=[] # guarda el número stirling (N,1:N) para consultarlo
        self.K = K0  # el número inicial de temas
        
        self.alpha = alpha # los temas principales
        self.beta = beta   # las palabras principales
        self.gamma = gamma # las tablas principales
        
        self.docs = docs # lista de documentos que incluyen a las palabras
        self.V = V # el número de palabras diferentes en el vocabulario
        
        self.z_m_n = {} # asignación de temas a documentos

        # número de temas asignados al tema z en el documento m
        self.n_m_z = numpy.zeros((len(self.docs), self.K)) 
        self.theta = numpy.zeros((len(self.docs), self.K))
        
        # número de veces que una palabra v se asigna a un tema z
        self.n_z_t = numpy.zeros((self.K, V)) 
        self.phi = numpy.zeros((self.K, V))
        
        # número total de palabras asignadas a un tema z
        self.n_z = numpy.zeros(self.K)   
        self.U1=[] # temas activos
        for i in range (self.K):
            self.U1.append(i)
        
        self.U0=[] # temas desactivados
        self.tau=numpy.zeros(self.K+1) +1./self.K
        for m, doc in enumerate(docs):    # initialización de estructuras de datos
            for n,t in enumerate(doc):
                #aleatoriamente se asigna un tema a una palabra y se incrementa el array de conteo
                z = numpy.random.randint(0, self.K) 
                self.n_m_z[m, z] += 1
                self.n_z_t[z, t] += 1
                self.n_z[z] += 1
                self.z_m_n[(m,n)]=z
        
    def inference(self,iteration):
        "Inferencia de HDP empleando asignación Direchlet con simplificación ILDA"
        
        for m, doc in enumerate(self.docs):
            for n, t in enumerate(doc):
          
            # se decrementa el conteo de la palabra t para el tema k_old
                k_old =self.z_m_n[(m,n)]
                self.n_m_z[m,k_old] -= 1
                self.n_z_t[k_old, t] -= 1
                self.n_z[k_old] -= 1

                p_z=numpy.zeros(self.K+1)
                # se hace muestreo con la función z de ILDA
                for kk in range (self.K): 
                    k=self.U1[kk]
                    p_z[kk]=(self.n_m_z[m,k] + self.alpha * self.tau[k])*(self.n_z_t[k,t]+self.beta)/(self.n_z[k]+self.V*self.beta)

                # se genera una nueva coordenada para un tema nuevo
                p_z[self.K]=(self.alpha*self.tau[self.K])/self.V 

                k_new = numpy.random.multinomial(1, p_z / p_z.sum()).argmax()

                if k_new==self.K:
                    # se incrementa el número de temas y arrays y se asigna al array un nuevo tema

                    self.z_m_n[(m,n)] = self.spawntopic(m,t) 
                    # se actualiza la tabla de distribución sobre temas
                    self.updatetau() 

                else :
                    # hace lo mismo que LDA
                    k=self.U1[k_new] 
                    self.z_m_n[(m,n)] = k
                    self.n_m_z[m,k] += 1
                    self.n_z_t[k, t] += 1
                    self.n_z[k] += 1
                
                # verifica si un tema no ha sido usado y reformula sus elementos
                if self.n_z[k_old]==0: 
                    self.U1.remove(k_old)
                    self.U0.append(k_old)
                    self.K -=1                    
                    self.updatetau()

        # print ('Iteration:',iteration,'\n','Number of topics:',self.K,'\n','Temas activos:',self.U1,'\n','Deactivated topics',self.U0)


    def spawntopic (self,m,t): # reformula los arrays de elementos para un nuevo tema
        if len(self.U0)>0: # si hay temas desactivados
            k=self.U0[0]
            self.U0.remove(k)
            self.U1.append(k)
            self.n_m_z[m,k]=1
            self.n_z_t[k,t]=1
            self.n_z[k]=1
            
            
        else:
            k=self.K # si hasta el momento no hay temas inactivos
            self.n_m_z=numpy.append(self.n_m_z,numpy.zeros([len(self.docs),1]),1)
            self.U1.append(k)
            self.n_m_z[m,k] = 1
            self.n_z_t=numpy.vstack([self.n_z_t,numpy.zeros(self.V)])
            self.n_z_t[k, t] = 1
            self.n_z=numpy.append(self.n_z,1)
            self.tau=numpy.append(self.tau,0)
        
        self.K +=1
        
        return k
    
    def stirling(self,nn): # guardar en una matriz los valores stirling (N, 1: N), única vez
        if len(self.alss)==0: 
            self.alss.append([])
            self.alss[0].append(1)
        if nn > self.maxnn:
            for mm in range (self.maxnn,nn):
                ln=len(self.alss[mm-1])+1
                self.alss.append([])
                
                for xx in range(ln) :
                    self.alss[mm].append(0)
                    if xx< (ln-1):
                        self.alss[mm][xx] += self.alss[mm-1][xx]*mm
                    if xx>(ln-2) :
                        self.alss[mm][xx] += 0
                    if xx==0 :
                        self.alss[mm][xx] += 0
                    if xx!=0 :
                        self.alss[mm][xx] += self.alss[mm-1][xx-1]

            self.maxnn=nn
        return self.alss[nn-1]
    
    
    
    def rand_antoniak(self,alpha, n):
        # Realiza muestreo mediante una distribution Antoniak
        ss = self.stirling(n)
        max_val = max(ss)
        p = numpy.array(ss) / max_val
        
        aa = 1
        for i, _ in enumerate(p):
            p[i] *= aa
            aa *= alpha
        
        p = numpy.array(p,dtype='float') / numpy.array(p,dtype='float').sum()
        return choice(range(1, n+1), p=p)
    
    def updatetau(self):  # actualiza tau mediante muestreo Antoniak
    
        m_k=numpy.zeros(self.K+1)
        for kk in range(self.K):
            k=self.U1[kk]
            for m in range(len(self.docs)):
                
                if self.n_m_z[m,k]>1 :
                    m_k[kk]+=self.rand_antoniak(self.alpha*self.tau[k], int(self.n_m_z[m,k]))
                else :
                    m_k[kk]+=self.n_m_z[m,k]
    
        T=sum(m_k)
        m_k[self.K]=self.gamma
        tt=numpy.transpose(numpy.random.dirichlet(m_k, 1))
        for kk in range(self.K):
            k=self.U1[kk]
            self.tau[k]=tt[kk]

        self.tau[self.K]=tt[self.K]



    def worddist(self):
        """Distribution tema-palabra, \phi en paper original de Blei  """
        return (self.n_z_t +self.beta)/ (self.n_z[:, numpy.newaxis]+self.V*self.beta),len(self.n_z)





        

### Función Principal

Se leen los archivos y se ponen en la lista _corpus_. Una vez realizadas las iteraciones se hacen todas las inferencias para obtener el número de temas _K0_.


In [0]:
def get_doc_content(content_list):
    content_string = ""
    for content in content_list:
        content_string = content_string + " " + str(content)

    return content_string

# se invoca el funcionamiento del programa que aplica HDP al corpus
if __name__ == "__main__":
    
    # iterar los archivos en la carpeta
    datasets_dir = "dataset/bbc/bbc/entertainment"
    
    abs_datasets_dir = os.path.abspath(datasets_dir)

    filenames = os.listdir("dataset/bbc/bbc/entertainment/")

    corpus = list()
    for filename in filenames:
        fname = abs_datasets_dir + "/" + filename
        if(filename == ".ipynb_checkpoints"):
            continue
        content = get_doc_content(codecs.open(fname , 'r', encoding='utf8').read().splitlines()) 
        corpus.append(content)

    iterations = 100 # el número de iteraciones para que converja el algoritmo
    voca = Vocabulary(excluds_stopwords=True) # encuentra las palabras únicas del corpus
    docs = [voca.doc_to_ids(doc) for doc in corpus] # cambia las palabras por sus identificadores
    HDP = HDP_gibbs_sampling(K0=20, alpha=0.5, beta=0.5, gamma=2, docs=docs, V=voca.size()) # inicializa HDP
    for i in range(iterations):
        HDP.inference(i)
    (d,len) = HDP.worddist() # encuentra la distribución de palabras por cada tema
    for i in range(len):
        ind = numpy.argpartition(d[i], -10)[-10:] # las 10 palabras top por cada tema
        for j in ind:
            print (voca[j],' ',end=""),
        print ()



